In [1]:
import os

os.chdir("/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/")

In [2]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType
from pyspark.sql.functions import col, from_json, to_date, avg, lit, to_timestamp
from pyspark.sql.window import Window
import time
import shutil
from pyspark.sql.functions import current_timestamp, col, unix_timestamp

from datetime import datetime, timedelta


import os

gcs_jar_path = os.path.abspath("config/gcs-connector-hadoop3-latest.jar")
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("hehee") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")\
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")\
    .config("spark.hadoop.fs.gs.auth.service.account.enable", "true")\
    .config("spark.hadoop.fs.gs.auth.service.account.json.keyfile", "config/key/btcanalysishust-495a3a227f22.json") \
    .config("spark.jars", gcs_jar_path) \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
    .config("spark.hadoop.fs.gs.project.id", "btcanalysishust")\
    .getOrCreate()


24/12/12 21:06:34 WARN Utils: Your hostname, MacBook-Pro-cua-My.local resolves to a loopback address: 127.0.0.1; using 192.168.1.6 instead (on interface en0)
24/12/12 21:06:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/spark-env/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/ibulmnie/.ivy2/cache
The jars for the packages stored in: /Users/ibulmnie/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-06971c58-befd-4521-8474-c8199c37b204;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:

In [15]:
df = spark.read.format("csv") \
    .option("header", "true") \
    .load("gs://indicator-crypto/sma_results/BTC/*")


In [16]:
df.show(truncate=False)

+-----------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|DATE                         |CLOSE   |SMA_5            |SMA_10           |SMA_20           |SMA_50           |SMA_100          |SMA_200          |
+-----------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|2024-12-12T21:10:14.356+07:00|100968.0|99550.54000000001|99453.99         |97630.995        |86770.63199999997|74535.0154       |68747.13929999998|
|2024-12-12T00:00:00.000+07:00|101552.7|99590.94         |98948.94         |97533.79650000001|86083.29419999997|74100.0238       |68584.66789999999|
|2024-12-11T00:00:00.000+07:00|101177.0|99260.38         |98379.0          |97378.11600000001|85399.84179999998|73675.8951       |68423.35274999999|
|2024-12-10T00:00:00.000+07:00|96655.1 |98999.55999999998|97907.84199999999|97038.9235       |84723.224799

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType
from pyspark.sql import functions as F

# Define schema
schema = StructType([
    StructField("BASE", StringType(), True),
    StructField("DATE", StringType(), True),
    StructField("OPEN", FloatType(), True),
    StructField("HIGH", FloatType(), True),
    StructField("LOW", FloatType(), True),
    StructField("CLOSE", FloatType(), True),
    StructField("VOLUME", FloatType(), True),
    StructField("YEAR", StringType(), True),
    StructField("MONTH", StringType(), True),
    StructField("__index_level_0__", StringType(), True),
    StructField("BTC_CLOSE", FloatType(), True),
    StructField("ETH_CLOSE", FloatType(), True),
    StructField("USDT_CLOSE", FloatType(), True),
    StructField("BNB_CLOSE", FloatType(), True),
    StructField("USDC_CLOSE", FloatType(), True),
    StructField("XRP_CLOSE", FloatType(), True),
    StructField("ADA_CLOSE", FloatType(), True),
    StructField("MATIC_CLOSE", FloatType(), True),
    StructField("DOGE_CLOSE", FloatType(), True),
    StructField("SOL_CLOSE", FloatType(), True),
    StructField("SMA5_BTC_CLOSE", FloatType(), True),
    StructField("SMA10_BTC_CLOSE", FloatType(), True),
    StructField("SMA20_BTC_CLOSE", FloatType(), True),
    StructField("SMA50_BTC_CLOSE", FloatType(), True),
    StructField("SMA100_BTC_CLOSE", FloatType(), True),
    StructField("SMA200_BTC_CLOSE", FloatType(), True),
])

# Read stream with the defined schema
df = spark.readStream.schema(schema).parquet("gs://indicator-crypto/sma_results/BTC_CLOSE")

# Convert the 'DATE' column to timestamp if it's a string type
df = df.withColumn("DATE", F.to_timestamp("DATE", "yyyy-MM-dd"))

# Write the stream to a sink (e.g., console, file, etc.)
query = df.writeStream.outputMode("append").format("console").start()

# Wait for the streaming to finish
query.awaitTermination()
